In [5]:
import numpy as  np
import cv2

In [ ]:
webcam = cv2.VideoCapture(0,cv2.CAP_DSHOW) 

while True:
    ret,current_frame = webcam.read()
    
    img_height = current_frame.shape[0]
    img_width = current_frame.shape[1]

    img_resize = cv2.resize(current_frame,(300,300))     # In caffe model input shape is 300x300
    img_blob = cv2.dnn.blobFromImage(img_resize,0.007843,(300,300),127.5)
    #recommended scale factor is 0.007843, width,height of blob is 300,300, mean of 255 is 127.5 

    class_labels = ["background", "aeroplane", "bicycle", "bird", "boat","bottle", "bus", "car", "cat", "chair","cow",
                    "diningtable","dog","horse","motorbike","person","pottedplant","sheep","sofa", "train","tvmonitor"]
    
    mobilenetssd = cv2.dnn.readNetFromCaffe('mobilenetssd.prototext','mobilenetssd.caffemodel')
    mobilenetssd.setInput(img_blob)
    obj_detections = mobilenetssd.forward()   # detection predictions by the model using forward()
    
    no_of_detections = obj_detections.shape[2]
            
    for i in np.arange(0, no_of_detections):
        prediction_confidence = obj_detections[0, 0, i, 2]
    
        if prediction_confidence >= 0.30:

            predicted_class_index = int(obj_detections[0, 0, i, 1])
            predicted_class_label = class_labels[predicted_class_index]

            #obtain the bounding box co-oridnates for actual image from resized image size
            bounding_box = obj_detections[0, 0, i, 3:7] * np.array([img_width, img_height, img_width, img_height])
            (start_x_pt, start_y_pt, end_x_pt, end_y_pt) = bounding_box.astype("int")


            predicted_class_label = "{}: {:.2f}%".format(class_labels[predicted_class_index], prediction_confidence * 100)
            cv2.rectangle(current_frame, (start_x_pt, start_y_pt), (end_x_pt, end_y_pt), (0,255,0), 2)
            cv2.putText(current_frame, predicted_class_label, (start_x_pt, start_y_pt-5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255), 1)

    
    cv2.imshow("xyz", current_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

webcam.release()
cv2.destroyAllWindows() 

--------------------------